In [191]:
import pandas as pd
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import log_loss
from sklearn.model_selection import train_test_split
import optuna
from optuna import trial, visualization
import os
import argparse
import optuna
from optuna_dashboard import run_server
from optuna.trial import TrialState
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
train_path = '/Users/egortuzarov/ai-masters-bigdata/train1000.txt'

In [215]:
numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]
fields = ["id", "label"] + numeric_features + categorical_features
read_table_opts = dict(sep="\t", names=fields, index_col=False)
df = pd.read_table(train_path, **read_table_opts)

In [216]:
y = df['label']
X = df.drop(['label', 'id'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [217]:
from sklearn.preprocessing import StandardScaler

numeric_features = ["if"+str(i) for i in range(1,14)]
categorical_features = ["cf"+str(i) for i in range(1,27)]
fields = ["id", "label"] + numeric_features + categorical_features
read_table_opts = dict(sep="\t", names=fields, index_col=False)
df = pd.read_table(train_path, **read_table_opts)
myImputer = SimpleImputer(missing_values=pd.NA, strategy= 'median')
df[numeric_features] = myImputer.fit_transform(df[numeric_features])
catImputer = SimpleImputer(missing_values=pd.NA, strategy= 'constant', fill_value='miss')
df[categorical_features] = catImputer.fit_transform(df[categorical_features])
df = pd.get_dummies(df, columns=categorical_features, drop_first=True)
scaler = StandardScaler()
df[numeric_features] = scaler.fit_transform(df[numeric_features])
topfeat = ['cf15_91f74a64', 'cf10_6320f08e', 'cf7_0aea30a9', 'cf13_3d3449d7',
       'cf11_53e86199', 'cf17_e5ba7672', 'cf18_87c9f30d', 'cf2_287130e0',
       'cf2_4f25e98b', 'cf10_6c47047a', 'cf13_b8a76289', 'cf11_c6c91669',
       'cf2_38a947a1', 'cf14_8ceecbc8', 'cf2_c1384774', 'cf17_d4bb7bd8',
       'cf11_0893f6c5', 'cf18_8e8b535e', 'cf10_7259dc52', 'cf26_aa5f0a15',
       'cf2_0a519c5c', 'cf25_9d93af03', 'cf23_be7c41b4', 'cf1_5bfa8ab5',
       'cf10_fbbf2c95', 'cf7_6cdb3998', 'cf1_241546e0', 'cf17_8efede7f',
       'cf6_6f6d9be8', 'if5']
y = df['label']
X = df[topfeat]
model = LogisticRegression()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)
score = log_loss(y_val, y_pred)

In [225]:
y = df['label']
X = df.drop(['label', 'id'], axis = 1)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [73]:
numeric_features = df.columns[0:6]
categorical_features = df.columns[7:26]

In [75]:
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ]
)
df_proc = Pipeline(steps=[('preprocessor', preprocessor)])

In [76]:
X = df_proc.fit_transform(df)

In [236]:
model = LogisticRegression()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train, y_train)
coefficients = model.coef_[0]

feature_importance = pd.DataFrame({'Feature': X.columns, 'Importance': np.abs(coefficients)})
feature_importance = feature_importance.sort_values('Importance', ascending=True)

In [243]:
top20 = feature_importance.tail(30)
topfeat = top20['Feature'].values

In [246]:
y = df['label']
X = df[topfeat]

In [255]:
from sklearn.metrics import log_loss

model = LogisticRegression()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
model.fit(X_train, y_train)
y_pred = model.predict_proba(X_val)
score = log_loss(y_val, y_pred)

In [256]:
score

0.5147255525490726

In [231]:
from sklearn.model_selection import KFold, cross_val_score
model = LogisticRegression()
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.1, random_state=42)
scores = np.array([])
for i in range(10):
    fold = KFold(n_splits=5, shuffle=True, random_state=i)
    scores_on_this_split = cross_val_score(
            estimator= model, X=X_train, y=y_train,
            cv=fold, scoring='neg_log_loss')
    scores = np.append(scores, scores_on_this_split)

array([-0.56774016, -0.45066666, -0.45589994, -0.52275801, -0.7177113 ,
       -0.5043644 , -0.59745519, -0.47226339, -0.60869651, -0.54627986,
       -0.55613674, -0.63150405, -0.5902443 , -0.4831049 , -0.55257093,
       -0.49195113, -0.58998855, -0.49597067, -0.63880743, -0.58669583,
       -0.5392686 , -0.61873568, -0.50438012, -0.57116343, -0.561163  ,
       -0.54276346, -0.49772906, -0.62513179, -0.41501375, -0.61889313,
       -0.54568693, -0.62244868, -0.5152746 , -0.50061028, -0.43323794,
       -0.60722172, -0.46034589, -0.5527714 , -0.55597163, -0.50533796,
       -0.60775884, -0.64769422, -0.47211629, -0.44392382, -0.52449327,
       -0.52923835, -0.56545557, -0.49279927, -0.54599223, -0.58125435])

In [248]:
def my_objective(trial, X_tr, X_val, y_tr, y_val):
    # Определение пространства поиска гиперпараметров
    params = {
        'tol' : trial.suggest_uniform('tol' , 1e-6 , 1e-3),
        'penalty': trial.suggest_categorical('penalty', ['l2']),
        'C': trial.suggest_loguniform('C', 1e-2, 1),
        'solver': trial.suggest_categorical('solver', ['lbfgs', 'saga'])
    }
    model = LogisticRegression(random_state=42, **params)
    model.fit(X_tr, y_tr)
    
    # предсказываем на валидации [/ кроссвалидации] и получаем скор - считаем метрику качества
    
    return model.score(X_val, y_val)

In [12]:
def my_objectivel1(trial, X_tr, X_val, y_tr, y_val):
    # Определение пространства поиска гиперпараметров
    params = {
        'penalty': trial.suggest_categorical('penalty', ['l1', 'elasticnet']),
        'C': trial.suggest_loguniform('C', 1e-2, 1),
        'solver': trial.suggest_categorical('solver', ['saga']),
        'l1_ratio' : trial.suggest_loguniform('l1_ratio' , 1e-4 , 1e-2)
    }
    model = LogisticRegression(random_state=42, **params)
    model.fit(X_tr, y_tr)
    
    # предсказываем на валидации [/ кроссвалидации] и получаем скор - считаем метрику качества
    
    return model.score(X_val, y_val)

In [249]:

study = optuna.create_study(sampler=None, direction='maximize')

from functools import partial
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Нужно сделать так, чтобы наша функция-objective в итоге принимала только аргумент trial.
# Поэтому зафиксируем заранее все остальные
obj_func = partial(my_objective, X_tr=X_tr, X_val=X_val, y_tr=y_tr, y_val=y_val)

study.optimize(obj_func, n_trials=1000, n_jobs=3)


[I 2024-04-21 16:57:57,178] A new study created in memory with name: no-name-9739c15d-07c9-495c-bd92-c5c9a1efc05d
/var/folders/cg/d2dr2x4j2gg4p1t5ks1tpvrr0000gn/T/ipykernel_1272/1254818902.py:4: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float instead.
  'tol' : trial.suggest_uniform('tol' , 1e-6 , 1e-3),
/var/folders/cg/d2dr2x4j2gg4p1t5ks1tpvrr0000gn/T/ipykernel_1272/1254818902.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 1e-2, 1),
/var/folders/cg/d2dr2x4j2gg4p1t5ks1tpvrr0000gn/T/ipykernel_1272/1254818902.py:6: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases

In [16]:
study = optuna.create_study(sampler=optuna.samplers.TPESampler())
X_tr, X_val, y_tr, y_val = train_test_split(X, y, test_size=0.2, random_state=42)
# Нужно сделать так, чтобы наша функция-objective в итоге принимала только аргумент trial.
# Поэтому зафиксируем заранее все остальные
obj_func = partial(my_objectivel1, X_tr=X_tr, X_val=X_val, y_tr=y_tr, y_val=y_val)
n_trials = 200
tpe_sampler = optuna.samplers.TPESampler(
    n_startup_trials=n_trials/2, # объем разведки. Всегда настраивайте!
    gamma=lambda n_trials: min(int(np.ceil(0.1 * n_trials)), 25), # не обязательно настраивать. По умолчанию топ-10%
    n_ei_candidates=13, # влияет на "точность шага"
)
study = optuna.create_study(sampler=tpe_sampler, direction='maximize')

study.optimize(obj_func, n_trials=n_trials)

[I 2024-04-21 14:11:14,079] A new study created in memory with name: no-name-6d04abed-c3c5-4f0e-a571-b121fa01473a
[I 2024-04-21 14:11:14,087] A new study created in memory with name: no-name-878fb397-914d-49b2-b16e-7a74567d5cd0
/var/folders/cg/d2dr2x4j2gg4p1t5ks1tpvrr0000gn/T/ipykernel_1272/3077633217.py:5: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'C': trial.suggest_loguniform('C', 1e-2, 1),
/var/folders/cg/d2dr2x4j2gg4p1t5ks1tpvrr0000gn/T/ipykernel_1272/3077633217.py:7: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'l1_ratio' : trial.suggest_loguniform('l1_ratio' , 1e-4 , 1e-2)
/Users/egortuzarov/anaconda3/envs/homecredit/lib/python3.11/site-packages/sklearn/linea

KeyboardInterrupt: 

In [81]:
study.best_params

{'tol': 0.0002231984825751999,
 'penalty': 'l2',
 'C': 0.1124135904298268,
 'solver': 'saga'}